# <font color="steelblue">Algoritmos Bagging avanzados: Bosques aleatorios (Random Forest)</font>

**Autoría**: 

*   Fernando Borrás (f.borras@umh.es)
*   Federico Botella (federico@umh.es)
*   Inés Hernández (ines.hernandezp@umh.es)
*   Mª Asunción Martínez Mayoral (asun.mayoral@umh.es)
*   Josep Moltó (j.molto@umh.es)
*   Javier Morales (j.morales@umh.es) 

Departamento de Estadística, Matemáticas e Informática. 

Universidad Miguel Hernández de Elche. 


**Financiación**: El material que aparece a continuación se ha desarrollado dentro del marco del proyecto UNIDIGITAL- IASAC.

<small><img src=https://raw.githubusercontent.com/ia4legos/MachineLearning/main/images/IASAC-UMH.png width="450" height="200"></small>

**Fecha última edición**: 26/05/2022

**Licencia**: <small><a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br /></small>

No olvides hacer una copia si deseas utilizarlo. Al usar estos contenidos, acepta nuestros términos de uso y nuestra política de privacidad. 


## <font color="steelblue">Configuración del cuaderno</font>

Para garantizar la funcionalidad completa de este cuaderno, es preciso ejecutar la celda de código a continuación.

In [ ]:
#@title <b><font color="steelblue" size="+1"> Configuración de cuaderno

# Cargamos módulos
from io import StringIO 
import sys
import numpy as np      # importamos numpy como np
import pandas as pd     # importamos pandas como pd
import math             # importamos módulo para cáculos matemáticos

# Esta línea configura matplotlib para mostrar las figuras incrustadas en el jupyter notebook
# Configuraciónde entorno gráfico
%matplotlib inline
import matplotlib.pyplot as plt # importamos matplotlib como plt
import seaborn as sns # importamos seaborn como sns
sns.set_style("ticks")
%config InlineBackend.figure_format = 'retina'

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

with Capturing() as output:
    print('Comenzamos....')

with Capturing(output) as output:
    # Librerías
    import os
    !pip install jupyterquiz
    from jupyterquiz import display_quiz
    import json
    import base64
    # Lectura ficheros json
    # A configurar pra cada cuaderno en función de las preguntas de autoevalaución
    for i in range(1,7):
      !wget {f"https://raw.githubusercontent.com/ia4legos/MachineLearning/main/autoeval/auto_50_{i}.json"}
    print("Cuaderno configurado")

if output[-1]=='Cuaderno configurado':
    print(output[-1])
else: 
    print(output[:-2])

Cuaderno configurado


# <font color="steelblue">Introducción</font>

**Descripción:** 

**Nivel de Formación:** 

**Recomendaciones antes de usarlo:** 

**Objetivos de aprendizaje:**

* 

## <font color="steelblue">Objetivos de aprendizaje</font>

- 

## <font color="steelblue">Contenidos</font>

1. 

# <font color="steelblue">Introducción al bosque aleatorio (Random Forest)</font>

Un modelo de **bosque aleatorio** está formado por un conjunto de árboles de decisión individuales, cada uno entrenado con una muestra ligeramente distinta de los datos de entrenamiento generada mediante bootstrapping. La predicción de una nueva observación se obtiene agregando las predicciones de todos los árboles individuales que forman el modelo.

Muchos métodos predictivos generan modelos globales en los que disponemos de una única ecuación o modelo de predicción. Sin embargo, en situaciones complejas con múltiples predictores, que interaccionan entre ellos de forma compleja y no lineal, es muy difícil encontrar un modelo predictivo los suficientemente preciso. Como ya hemos visto anteriormente los árboles de decisión nos permiten obtener un modelo con el que odemos manejar de forma sencilla relaciones complejas entre las posibles predictoras

Sin embargo, como ya hemos visto la utilización de los árboles de decisión no está exenta de dificultades y por ese motivo se introduce aqui el algortimo de bosque aleatorio que es un método de conjunto bagging que nos permite mejorar la capacidad predictiva de los árboles de decisión individuales.

Entre la ventajas del suso de este tipo de algortimo podemos destacar:

* Son capaces de seleccionar predictores de forma automática.
* Pueden aplicarse a problemas de regresión y clasificación.
* Al tratarse de métodos no paramétricos, no es necesario que se cumpla ningún tipo de distribución específica.
* Por lo general, requieren mucha menos limpieza y pre procesado de los datos en comparación a otros métodos de aprendizaje estadístico (por ejemplo, no requieren estandarización).
* No se ven muy influenciados por outliers.
* Son muy útiles en la exploración de datos, permiten identificar de forma rápida y eficiente las variables (predictores) más importantes.
* Gracias al Out-of-Bag Error puede estimarse su error de validación sin necesidad de recurrir a estrategias computacionalmente costosas como la validación cruzada. 

Entre las desvantajas podemos destacar:

* Al combinar múltiples árboles, se pierde la interpretabilidad que tienen los modelos basados en un único árbol.
* Cuando tratan con predictores continuos, pierden parte de su información al categorizarlas en el momento de la división de los nodos.
* Por la forma de construcción de los árboles de decisión los predictores continuos o predictores cualitativos con muchos niveles tienen mayor probabilidad de contener, solo por azar, algún punto de corte óptimo, por lo que suelen verse favorecidos en la creación de los árboles.
* No son capaces de extrapolar fuera del rango de los predictores observado en los datos de entrenamiento.

# <font color="steelblue">Algoritmo de bosque aleatorio</font>

Antes de presentar el algoritmos específico de bosque aleatorio es necesario conocer como funciona el proceso de bagging para un único árbol de decisión. Dicho algoritmo se organiza en tres pasos:

1. Generar  𝐵  pseudo-training sets mediante bootstrapping a partir de la muestra de entrenamiento original.
2. Entrenar un árbol con cada una de las  𝐵  muestras del paso 1. Cada árbol se crea sin apenas restricciones y no se somete a pruning, por lo que tiene varianza alta pero poco sesgo. En la mayoría de casos, la única regla de parada es el número mínimo de observaciones que deben tener los nodos terminales. El valor óptimo de este hiperparámetro puede obtenerse comparando el out of bag error o por validación cruzada.
3. Para cada una de la muestras de validación, se obtiene la predicción en cada uno de los 𝐵  árboles. El valor final de la predicción se obtiene como la media de las  𝐵  predicciones en el caso de variables cuantitativas y como la clase predicha más frecuente (moda) para variables cualitativas.

En el algoritmo descrito, el número de árboles creados no es un hiperparámetro crítico en cuanto a que, por mucho que se incremente el número, no se aumenta el riesgo de overfitting. Alcanzado un determinado número de árboles, la reducción de test error se estabiliza. A pesar de ello, cada árbol ocupa memoria, por lo que no conviene almacenar más de los necesarios.

El algoritmo de Random Forest es una modificación del proceso de bagging anterior que consigue mejorar los resultados gracias a que considera árboles lo más independientes posible. 

Supóngase un set de datos en el que hay un predictor muy influyente, junto con otros moderadamente influyentes. En este escenario, todos o casi todos los árboles creados en el proceso de bagging estarán dominados por el mismo predictor y serán muy parecidos entre ellos. Como consecuencia de la alta correlación entre los árboles, el proceso de bagging apenas conseguirá disminuir la varianza y, por lo tanto, tampoco mejorar el modelo. Random forest evita este problema haciendo una selección aleatoria de  $𝑚$  predictores antes de evaluar cada división. De esta forma, un promedio de  $(𝑝−𝑚)/𝑝$  divisiones no contemplarán el predictor influyente, permitiendo que otros predictores puedan ser seleccionados. Añadiendo este paso extra se consigue decorrelacionar los árboles todavía más, con lo que su agregación consigue una mayor reducción de la varianza. Algunas recomndaciones para la selección de $m$ son:

* La raíz cuadrada del número total de predictores para problemas de clasificación:  

$$m \approx \sqrt{p}$$

* Un tercio del número de predictores para problemas de regresión:

$$m \approx p/3$$  

* Si los predictores están muy correlacionados, valores pequeños de  $𝑚$  consiguen mejores resultados.

# <font color="steelblue">Predicción mediante bosque aleatorio</font>

Para realizar la predicción de un bosque aleatorio utilizamos el principio de bagging, de forma que, una vez determinamos el nod terminal al que es asinado la observación apredecir en cada uno de los árboles, utilizamos las observaciones contenidas en dicho nodo terminal para la predicción individidual de cada uno de ellos. Si estamos en un modeo de regresión obtnemos la media de todas las observaciones del nodo terminal en cada árbol, mientras que si estamos en un problema de clasificación actuamos mediante el voto por mayoría.

Una vez obtenemos las predicciones individuales la predicción conjunta se obtiene a partir de la media o de la categoría más frecuente de todas ellas en función de que estemos en un problema de predicción o clasificación.

Sin embargo, en los problemas de regresión la predicción de un árbol de regresión puede verse como una variante de vecinos cercanos en la que, solo las observaciones que forman parte del mismo nodo terminal que la observación predicha, tienen influencia. Siguiendo esta aproximación, la predicción del árbol se define como la media ponderada de todas las observaciones de entrenamiento, donde el peso de cada observación depende únicamente de si forma parte o no del mismo nodo terminal, es decir, definimos los pesos del árbol j como un vector de $n$ componentes donde cada una de las componenetes toma el valor $w_j = 1/n_j$  si la observación peetenece al nodo terminal $j$ con $n_j$ observaciones, y 0 en otro caso. Para el bosque aletorio esto equivale a la media ponderada de todas las observaciones, empleando como pesos  la media de los vectores de pesos de los $M$ árboles considerados, es decir,

$$\hat{w}=\frac{\sum_{i=1}^{M}  w_i}{M}$$

$$y_{pred} = \sum_{i=1}^n \hat{w}_i y_i$$

# <font color="steelblue">Importancia de los predictores</font>

Si bien es cierto que el bosque aleatorio consigue mejorar la capacidad predictiva en comparación a los modelos basados en un único árbol, esto tiene un coste asociado, la interpretabilidad del modelo se reduce. Al tratarse de una combinación de múltiples árboles, no es posible obtener una representación gráfica sencilla del modelo y no es inmediato identificar de forma visual que predictores son más importantes. Sin embargo, se han desarrollado nuevas estrategias para cuantificar la importancia de los predictores que hacen de los modelos de bosque aleatorio una herramienta muy potente, no solo para predecir, sino también para el análisis exploratorio. Dos de estas medidas son: importancia por permutación e impureza de nodos.

## <font color="steelblue">Importancia por permutación</font>

Identifica la influencia que tiene cada predictor sobre una determinada métrica de evaluación del modelo (estimada por out-of-bag error o validación cruzada). El valor asociado con cada predictor se obtiene de la siguiente forma:

1. Crear el conjunto de árboles que forman el modelo.

2. Calcular una determinada métrica de error (mse, classification error, ...). Este es el valor de referencia ($𝑒𝑟𝑟_0$).

3. Para cada predictor $𝑗$:

  * Permutar en todos los árboles del modelo los valores del predictor $𝑗$ manteniendo el resto constante.

  * Recalcular la métrica tras la permutación, llámese ($𝑒𝑟𝑟_𝑗$).

  * Calcular el incremento en la métrica debido a la permutación del predictor $𝑗$

$$\%I_𝑗=100*\frac{err_j-err_0}{err_0}$$

Si el predictor permutado estaba contribuyendo al modelo, es de esperar que el modelo aumente su error, ya que se pierde la información que proporcionaba esa variable. El porcentaje en que se incrementa el error debido a la permutación del predictor $𝑗$  puede interpretarse como la influencia que tiene $𝑗$  sobre el modelo. Algo que suele llevar a confusiones es el hecho de que este incremento puede resultar negativo. Si la variable no contribuye al modelo, es posible que, al reorganizarla aleatoriamente, solo por azar, se consiga mejorar ligeramente el modelo, por lo que  $(𝑒𝑟𝑟_𝑗−𝑒𝑟𝑟_0)$ es negativo. A modo general, se puede considerar que estas variables tiene una importancia próxima a cero.

Aunque esta estrategia suele ser la más recomendado, cabe tomar algunas precauciones en su interpretación. Lo que cuantifican es la influencia que tienen los predictores sobre el modelo, no su relación con la variable respuesta. ¿Por qué es esto tan importante? Supóngase un escenario en el que se emplea esta estrategia con la finalidad de identificar qué predictores están relacionados con el peso de una persona, y que dos de los predictores son: el índice de masa corporal (IMC) y la altura. Como IMC y altura están muy correlacionados entre sí (la información que aportan es redundante), cuando se permute uno de ellos, el impacto en el modelo será mínimo, ya que el otro aporta la misma información. Como resultado, estos predictores aparecerán como poco influyentes aun cuando realmente están muy relacionados con la variable respuesta. Una forma de evitar problemas de este tipo es, siempre que se excluyan predictores de un modelo, comprobar el impacto que tiene en su capacidad predictiva.

## <font color="steelblue">Incremento de la pureza de los nodos</font>

Cuantifica el incremento total en la pureza de los nodos debido a divisiones en las que participa el predictor (promedio de todos los árboles). La forma de calcularlo es la siguiente: en cada división de los árboles, se registra el descenso conseguido en la medida empleada como criterio de división (índice Gini, mse entropía, ...). Para cada uno de los predictores, se calcula el descenso medio conseguido en el conjunto de árboles que forman el conjunto. Cuanto mayor sea este valor medio, mayor la contribución del predictor en el modelo.

# <font color="steelblue">Hiperparámetros relevantes en el bosque aleatorio</font>

Del conjunto de hiperparámetros que se pueden modificar en el bosque aleatorio los dos más interesantes son el número de árboles considerados y el númeo máximo de predictoras usadas en la construcción de cada árbol. 

Lo habitual es proceder de forma individual estudiando la influencia de cada uno de los hiperparámetros respecto de la capacidad predictiva del modelo utilizando el out of bag score (aunque se puede configurar el algoritmo para utilizar otra). Esas curvas de influencia nos permiten deteminar la evolución del error del modelo con respecto a es hiperparámetro y obtener así el conjunto óptimo de valores. 

Sin embargo, aunque el análisis individual de los hiperparámetros es útil para entender su impacto en el modelo e identificar rangos de interés, la búsqueda final no debe hacerse de forma secuencial, ya que cada hiperparámetro interacciona con los demás. Es preferible recurrir a grid search o random search para analizar varias combinaciones de hiperparámetros. Los dos métodos más habituales son el grid search basado en el out of bag o el grid search basado en validación cruzada.

# <font color="steelblue">Codificación de predictoras cualitativas</font>


Los modelos basados en árboles de decisión, entre ellos Random Forest, son capaces de utilizar predictores categóricos en su forma natural sin necesidad de convertirlos en variables dummy mediante one hot encoding. Sin embargo, en la práctica, depende de la implementación que tenga la librería o software utilizado. Esto tiene impacto directo en la estructura de los árboles generados y, en consecuencia, en los resultados predictivos del modelo y en la importancia calculada para los predictores.

Entre las dificulatdes más relvantes al utilizar one hot encoding se pueden destacar:

* El entrenamiento de los modelos es mas costos cuando se aplica one hotencoding debido al aumento de dimensionalidad al crear las nuevas variables dummy, obligando a que el algoritmo tenga que analizar muchos más puntos de división.
* Al convertir una variable categórica en múltiples variables dummy su importancia queda diluida, dificultando que el modelo pueda aprender de ella y perdiendo así capacidad predictiva. Este efecto es mayor cuantos más niveles tiene la variable original.
* Al diluir la importancia de los predictores categóricos, estos tienen menos probabilidad de ser seleccionada por el modelo, lo que desvirtúa las métricas que miden la importancia de los predictores.

Por el momento, en scikit-learn es necesario hacer one-hot-encoding para convertir las variables categóricas en variables dummy si deseamos usar random forest. La implementación de `H2O` sí permite utilizar directamente variables categóricas.


# <font color="steelblue">Comparación Random Forest y Gradient Boosting Trees</font>


Random Forest se compara con frecuencia con otro algoritmo basado en árboles, Gradient Boosting Trees. Ambos métodos son muy potentes y la superioridad de uno u otro depende del problema al que se apliquen. Algunos aspectos a tener en cuenta son:

* Gracias al out-of-bag error, el método de Random Forest no necesita recurrir a validación cruzada para la optimización de sus hiperparámetros. Esto puede ser una ventaja muy importante cuando los requerimientos computacionales son limitantes. Esta característica también está presente en Stochastic Gradient Boosting pero no en AdaBoost y Gradient Boosting.

* Random forest tiene menos hiperparámetros, lo que hace más sencillo su correcta implementación.

* Si existe una proporción alta de predictores irrelevantes, Random Forest puede verse perjudicado, sobre todo a medida que se reduce el número de predictores ($𝑚$) evaluados. 

* En Random Forest, cada modelo que forma el ensemble es independiente del resto, esto permite que el entrenamiento sea paralelizable.

* Random Forest no sufre problemas de overfitting por muchos árboles que se agreguen.

* Si se realiza una buena optimización de hiperparámetros, Gradient Boosting Trees suele obtener resultados ligeramente superiores.

# <font color="steelblue">Aplicaciones</font>




# <font color="steelblue">Referencias y enlaces de interés</font>





Árboles de decisión con Python: regresión y clasificación by Joaquín Amat Rodrigo, available under a Attribution 4.0 International (CC BY 4.0) at https://www.cienciadedatos.net/documentos/py07_arboles_decision_python.html

Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

Géron, A. (2017). Hands-on machine learning with Scikit-Learn and TensorFlow : concepts, tools, and techniques to build intelligent systems. Sebastopol, CA: O'Reilly Media. ISBN: 978-1491962299


https://nyandwi.com/machine_learning_complete/

